In [ ]:
from graphframes.examples import Graphs
from pyspark.sql.functions import *
#import pixiedust
#from pixiedust.display import *
sc_conf=sc.getConf()
sc_conf.set('spark.yarn.executor.memoryOverhead','65536').set('spark.yarn.driver.memoryOverhead','32768')

In [ ]:
#pixiedust.installPackage("http://10.179.60.112/home/7222115/graphframes-0.5.0-spark2.1-s_2.11.jar")

In [ ]:
# Create a Vertex DataFrame with unique ID column "id"
v = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame
from graphframes import *
g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

# Query: Count the number of "follow" connections in the graph.
g.edges.filter("relationship = 'follow'").count()

# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

In [ ]:
from graphframes.examples import Graphs
l = Graphs(sqlContext).friends()  # Get example graph

results = l.shortestPaths(landmarks=["a", "d"])
results.select("id", "distances").show()

In [ ]:
from graphframes.examples import Graphs
l = Graphs(sqlContext).friends()  # Get example graph

results = l.triangleCount()
results.select("id", "count").show()

In [ ]:
display(g)

In [ ]:
sqlContext.sql("show databases").show(200,False)

In [ ]:
for i in sqlContext.sql("show tables in orbiz_warehouse").collect():
    #sqlContext.sql("SELECT * from orbiz_warehouse."+i.tableName)
    print i.tableName
    sqlContext.sql("SHOW COLUMNS from orbis_warehouse."+i.tableName).show(200, False)
    

In [ ]:
sqlContext.sql("SELECT subsidiary_bvd_id,shareholder_bvd_id,guo_25,guo_50,subsidiary_independence_indicator,direct_percentage,direct_percentage_only_figures,total_percentage,source,type_of_relation,activearchived FROM orbis_warehouse.links_current").show(2000,False)

In [ ]:
sqlContext.sql("SELECT subsidiary_bvd_id,shareholder_bvd_id,subsidiary_independence_indicator,direct_percentage,direct_percentage_only_figures,total_percentage FROM orbis_warehouse.links_current WHERE subsidiary_bvd_id='GB00048839' and shareholder_bvd_id='CA20167NC'").show(200)

In [ ]:
sqlContext.sql("SELECT * FROM orbis_warehouse.industry_classifications").show(200)

In [ ]:
sqlContext.sql("SELECT * FROM orbis_warehouse.entities").show(200,False)

In [ ]:
for i in sqlContext.sql("show tables in orbis_parquet").collect():
    print i.tableName
    sqlContext.sql("SHOW COLUMNS from orbis_parquet."+i.tableName).show(200,False)

In [ ]:
for i in sqlContext.sql("show tables in interim_tables")\
.collect():
    #sqlContext.sql("SELECT * from orbiz_warehouse."+i.tableName)
    print i.tableName
    sqlContext.sql("SHOW COLUMNS from interim_tables."+i.tableName).show(200, False)
    

In [ ]:
query= "SELECT shareholder_bvd_id,subsidiary_bvd_id,shareholder_independence_indicator,guo_25,guo_50, \
        subsidiary_independence_indicator,direct_percentage,direct_percentage_only_figures,total_percentage,total_percentage_only_figures \
        FROM orbis_warehouse.links_current \
        WHERE shareholder_bvd_id<>subsidiary_bvd_id and activearchived='active' and CAST(direct_percentage_only_figures AS int)>49" 

In [ ]:
big_DF=sqlContext.sql(query)

In [ ]:
big_DF

In [ ]:
big_DF.count() #345567102

In [ ]:
big_DF.take(1000)

In [ ]:
#vertices=sqlContext.sql("SELECT distinct(shareholder_bvd_id) from big_DF union SELECT distinct(subsidiary_bvd_id) from big_DF")
#sqlContext.read.parquet("http://")
shares=big_DF.select('shareholder_bvd_id').distinct()
subs=big_DF.select('subsidiary_bvd_id').distinct()
entities=sqlContext.sql('SELECT bvd_id_of_the_subsidiary_or_shareholder,entity_type as sh_entity_type FROM orbis_parquet.entities')
contact=sqlContext.sql('SELECT bvd_id_number,name_native,name_internat,country,country_iso_code  FROM orbis_parquet.contact_info')
_vertices=shares.union(subs).selectExpr('shareholder_bvd_id as id')
__vertices=_vertices.join(entities,_vertices.id==entities.bvd_id_of_the_subsidiary_or_shareholder,'left_outer')
vertices=__vertices.join(contact,__vertices.id==contact.bvd_id_number,'left_outer').select('id','sh_entity_type','name_native','name_internat','country','country_iso_code')

In [ ]:
shares.count()#77217721

In [ ]:
subs.count()#63045082

In [ ]:
vertices

In [ ]:
vertices.count() #208465222  140262804

In [ ]:
vertices.show(200)

In [ ]:
vertices_f=vertices.filter("id like 'GB%' or id like 'NL%' or id like 'IE%' or id like 'US%'")

In [ ]:
vertices_f.count()

In [ ]:
edges=big_DF.selectExpr('shareholder_bvd_id as src','subsidiary_bvd_id as dst','guo_25','guo_50','direct_percentage','direct_percentage_only_figures','total_percentage','total_percentage_only_figures')

In [ ]:
edges.count() #345567102

In [ ]:
edges_=edges.select("*").groupBy('src','dst','guo_25','guo_50','direct_percentage','direct_percentage_only_figures','total_percentage','total_percentage_only_figures').count()

In [ ]:
edges_

In [ ]:
edges_.count() #178375133 #108301140 126790241

In [ ]:
edges_f=edges_.filter("(src like 'GB%' or src like 'NL%' or src like 'IE%' or src like 'US%') and (dst like 'GB%' or dst like 'NL%' or dst like 'IE%' or dst like 'US%')")

In [ ]:
edges_f.count() #15059702

In [ ]:
from graphframes import GraphFrame

In [ ]:
orbis_GF=GraphFrame(vertices_f,edges_f).persist()

In [ ]:
orbis_GF

In [ ]:
orbis_GF.inDegrees.show(100)

In [ ]:
from pyspark.sql.functions import *

In [ ]:
orbis_GF.degrees.sort(desc("degree")).show(1000)#|    GB00617987| 19561520|-

In [ ]:
big_DF.filter("shareholder_bvd_id='GB00617987'").show(1000)

In [ ]:
big_DF.filter("shareholder_bvd_id = 'GB00617987' or subsidiary_bvd_id = 'GB00617987'").show(1000)

In [ ]:
test=big_DF.limit(100)

In [ ]:
comps=orbis_GF.stronglyConnectedComponents(maxIter=2)###

In [ ]:
comps

In [ ]:
comps.select('id','component').groupBy('component').agg(count("id").alias("count")).orderBy(desc("count")).count()#15659583

In [ ]:
comps.select('id','component').groupBy('component').agg(count("id").alias("count")).orderBy(desc("count")).show(1000)

In [ ]:
GT2=comps.select('id','component').groupBy('component').agg(count("id").alias("count")).filter('count>2').orderBy(desc("count"))  

In [ ]:
GT2_id=comps.join(GT2,comps.component==GT2.component,'inner').select('id',comps.component)

In [ ]:
GT2_id.count

In [ ]:
IDl=list(set([x.id for x in GT2_id.select('id').collect()]))

In [ ]:
edges_f2=edges_f.filter(edges_f.src.isin(IDl) | edges_f.dst.isin(IDl))

In [ ]:
orbis_GF2=GraphFrame(orbis_GF.vertices,edges_f2).persist()

In [ ]:
orbis_GF.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)").show(12,False)

In [ ]:
big_DF.filter("shareholder_bvd_id='DE8330233188'").show(1000,False)

In [ ]:
edges_.filter("src='DE8330233188'").show()

In [ ]:
edges_.filter("src='DE8330247688'").show()

In [ ]:
results_2h=orbis_GF.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)").filter("a.id like 'GB%'")

In [ ]:
results_2h.show(100,False)

In [ ]:
orbis_GF.bfs(fromExpr="id='GB01417162'",toExpr="id='FR572093920'",maxPathLength=5).show()

In [ ]:
edges_.filter("src='GB01417162'").show(1000)

In [ ]:
orbis_GF.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)").filter("a.id like 'GB02805337%'").show(100,False)

In [ ]:
orbis_GF.bfs(fromExpr="id='GB02805337'",toExpr="id='GB00048839'",maxPathLength=5).show(1000,False)

In [ ]:
orbis_GF.shortestPaths(landmarks=['GB02805337']).select('id','distances').show(100,False)#bad!

In [ ]:
communities = orbis_GF.labelPropagation(maxIter=5).select("id", "label").groupBy("label").agg(F.count("id"))#.alias("count")).orderBy(desc("count"))

# Visualisations

In [ ]:
from IPython.core .display import HTML,display
from string import Template


In [ ]:
ires=orbis_GF.find("(a)-[ab]->(b); (b)-[bc]->(c);(c)-[cd]->(d); (d)-[da]->(a)").collect()#.filter("a.id like 'GB01417162%'").take(1000)

In [ ]:
ires=orbis_GF2.find("(a)-[ab]->(b)").collect()

In [ ]:
ires.__str__()

In [ ]:
type(ires),type(ires[0]),ires[0]

In [ ]:
_ires0=ires[0]

In [ ]:

s=_ires0.asDict()

In [ ]:
_ires0.a.id

In [ ]:
[i.asDict() for i in ires]

In [ ]:
import itertools
ed_=[{k:v for k,v in i.iteritems() if len(k)==2} for i in [i.asDict() for i in ires]]
links_=list(itertools.chain.from_iterable([i for i in [i.values() for i in ed_]]))
links__=[dict([('source',i['src'].encode('utf-8')),('target',i['dst'].encode('utf-8')),('direct_percentage',i['direct_percentage_only_figures'].encode('utf-8')),('count',i['count']),('guo_50',(i['guo_50'] or '').encode('utf-8'))]) for i in set(links_)]

In [ ]:
ed_=[{k:v for k,v in i.iteritems() if len(k)==1} for i in [i.asDict() for i in ires]]
nodes_=list(itertools.chain.from_iterable([i for i in [i.values() for i in ed_]]))
nodes__=[dict([('id',i['id'].encode('utf-8')),('sh_entity_type',i['sh_entity_type'].encode('utf-8')),('name_native',(i['name_native'] or '').encode('utf-8'))]) for i in set(nodes_)]

In [ ]:
ed_

In [ ]:
links_

In [ ]:

links__

In [ ]:
nodes__

In [ ]:
mynodes=dict([(i['id'],i) for i in nodes__])

In [ ]:
mynodes['GB01417162']#['name_native']

In [ ]:
len(ed_),len(links_),len(links__)

In [ ]:
b=open('/home/7222115/d3.v3.min.js','r').read()
src=Template("""

<style>

#render_port{

background-color: #413C58;
}
path.link {
  fill: none;
  stroke: #49cc90;
  stroke-width: 1.5px;
}

circle {
  fill: #ccc;
  stroke: #fff;
  stroke-width: 2.5px;
}

text {
  fill: #ffff;
  font: 10px sans-serif;
  pointer-events: none;
}

</style>
<div id='render_port'></div>
<script>$myd3</script>
<script>

// get the data
links=$mylinks
myNodes=$mynodes
var nodes = {};

// Compute the distinct nodes from the links.
links.forEach(function(link) {
    link.source = nodes[link.source] || 
        (nodes[link.source] = {name: link.source, entity_type: link.sh_entity_type});
    
});

links.forEach(function(link) {
    link.target = nodes[link.target] || 
        (nodes[link.target] = {name: link.target});
    
});


var width = 960,
    height = 1000;



function isConnected(a, b) {
  return isConnectedAsTarget(a, b) || isConnectedAsSource(a, b) || a.index === b.index;
}

function isConnectedAsSource(a, b) {
  return linkedByIndex[`${a.index},${b.index}`];
}

function isConnectedAsTarget(a, b) {
  return linkedByIndex[`${b.index},${a.index}`];
}

function isEqual(a, b) {
  return a.index === b.index;
}

const mouseOverFunction = function (d) {
  const circle = d3.select(this);

  node
    .transition(500)
      .style('opacity', o => {
        const isConnectedValue = isConnected(o, d);
        if (isConnectedValue) {
          return 1.0;
        }
        return 0.2
      });
      

  path
    .transition(500)
      .style('stroke-opacity', o => (o.source === d || o.target === d ? 1 : 0.2))
      .transition(500)
      .attr('marker-end', o => (o.source === d || o.target === d ? 'url(#end)' : 'url()'));

  circle
    .transition(500)
      .attr('r', () => 1.4 *5);
};

const mouseOutFunction = function () {
  const circle = d3.select(this);

  node
    .transition(500);

  path
    .transition(500);

  circle
    .transition(500)
      .attr('r', 5);
 
svg.selectAll(".path").style("stroke-opacity", 1);

};

var force = d3.layout.force()
    .nodes(d3.values(nodes))
    .links(links)
    .size([width, height])
    .linkDistance(60)
    .charge(-50)
    .on("tick", tick)
    .start();

var svg = d3.select("#render_port").append("svg")
    .attr("width", width)
    .attr("height", height);

// build the arrow.
svg.append("svg:defs").selectAll("marker")
    .data(["end"])      // Different link/path types can be defined here
  .enter().append("svg:marker")    // This section adds in the arrows
    .attr("id", String)
    .attr("viewBox", "0 -5 10 10")
    .attr("refX", 15)
    .attr("refY", -1.5)
    .attr("markerWidth", 6)
    .attr("markerHeight", 6)
    .attr("orient", "auto")
  .append("svg:path")
    .attr("d", "M0,-5L10,0L0,5");

// add the links and the arrows
var path = svg.append("svg:g").selectAll("path")
    .data(force.links())
  .enter().append("svg:path")
//    .attr("class", function(d) { return "link " + d.type; })
    .attr("class", "link")
    .attr("marker-end", "url(#end)");
    
    path.append("title")
        .text(function(d) { return d.direct_percentage; });

// define the nodes
var node = svg.selectAll(".node")
    .data(force.nodes())
  .enter().append("g")
    .attr("class", "node")
    .call(force.drag);

// add the nodes
node.append("circle")
    .attr("r", 5)
     .on('mouseover', mouseOverFunction)
    .on('mouseout', mouseOutFunction);

// add the text 
node.append("text")
    .attr("x", 12)
    .attr("dy", ".35em")
    .text(function(d) { return d.name; });
    
node.append("title")
      .text(function(d) { return myNodes[d.name]['name_native']+"\\n"+myNodes[d.name]['sh_entity_type']; });

node.on("click", function(d) { d.fixed = true; });
node.on("dblclick", function(d) { d.fixed = false; });

let linkedByIndex = {};
links.forEach((d) => {
  linkedByIndex[`${d.source.index},${d.target.index}`] = true;
});

// add the curvy lines
function tick() {
    path.attr("d", function(d) {
        var dx = d.target.x - d.source.x,
            dy = d.target.y - d.source.y,
            dr = Math.sqrt(dx * dx + dy * dy);
        return "M" + 
            d.source.x + "," + 
            d.source.y + "A" + 
            dr + "," + dr + " 0 0,1 " + 
            d.target.x + "," + 
            d.target.y;
    });

    node
        .attr("transform", function(d) { 
  	    return "translate(" + d.x + "," + d.y + ")"; });
};

</script>

""")

In [ ]:
src_=src.safe_substitute({'myd3':b,'mylinks':links__,'mynodes':mynodes})

In [ ]:
html_=HTML(src_)

In [ ]:
html_

In [ ]:
#src__=Template("""<script>var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));
#win.document.body.innerHTML ='<div>$mysrc</div>'
#</script>""")

In [ ]:
#HTML(src__.safe_substitute({'mysrc':src_}))

In [ ]:
src__.safe_substitute({'mysrc':src_})

In [ ]:
gg=comps.filter("component='515396096104'")

In [ ]:
gg

In [ ]:
gg.show()